In [15]:
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

import supervision as sv
import cv2

from mediapipe_model_maker import object_detector

import random
import shutil

In [ ]:
# Define the path to the folder containing the individual JSON files
folder_path = "android_figurine/Blue_ball"
output_file = "android_figurine/Blue_ball/labels.json"

# Define the category mapping for blue_ball
category_mapping = {
    "blue_ball": 1  # blue_ball with category_id 1
}

# Initialize the combined data structure
combined_data = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 0, "name": "background"},
        {"id": 1, "name": "blue_ball"}
    ]
}

# Initialize counters for image and annotation IDs
image_id_counter = 0
annotation_id_counter = 0

# Iterate through each JSON file in the folder
for json_file in os.listdir(folder_path):
    if json_file.endswith(".json"):
        with open(os.path.join(folder_path, json_file), "r") as file:
            data = json.load(file)
            
            # Check if data is a list
            if isinstance(data, list):
                # Process the list of dictionaries
                for entry in data:
                    # Get image file name
                    image_file_name = entry.get("image", None)
                    
                    if not image_file_name:
                        print(f"Skipping file {json_file}: 'image' key not found.")
                        continue
                    
                    # Add image information
                    combined_data["images"].append({
                        "id": image_id_counter,
                        "file_name": image_file_name
                    })
                    
                    # Process annotations
                    annotations = entry.get("annotations", [])
                    for annotation in annotations:
                        label = annotation.get("label", None)
                        coordinates = annotation.get("coordinates", None)
                        
                        if not label or not coordinates:
                            print(f"Skipping annotation in {json_file}: 'label' or 'coordinates' key not found.")
                            continue
                        
                        # Map the label (always "blue_ball") to category_id 1
                        category_id = category_mapping.get(label, 0)  # Should map to 1 (blue_ball)
                        
                        # Extract bounding box in COCO format [x_min, y_min, width, height]
                        bbox = [
                            coordinates["x"] - coordinates["width"] / 2,  # x_min
                            coordinates["y"] - coordinates["height"] / 2,  # y_min
                            coordinates["width"],  # width
                            coordinates["height"]  # height
                        ]
                        
                        # Append the annotation to combined data
                        combined_data["annotations"].append({
                            "image_id": image_id_counter,
                            "bbox": bbox,
                            "category_id": category_id
                        })
                        
                        # Increment annotation ID
                        annotation_id_counter += 1
                    
                    # Increment image ID
                    image_id_counter += 1
            else:
                print(f"Skipping file {json_file}: Expected a list but got {type(data)}")

# Save the combined JSON file
with open(output_file, "w") as output:
    json.dump(combined_data, output, indent=4)

print(f"Combined JSON saved to {output_file}")

In [14]:
#Augmentation here


In [ ]:
with open(os.path.join('android_figurine/Blue_ball', "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")


In [18]:
# Paths
original_images_path = "android_figurine/Blue_ball"
train_images_path = "android_figurine/Blue_ball/train"
val_images_path = "android_figurine/Blue_ball/val"
labels_path = "android_figurine/Blue_ball/labels.json"

# Load labels.json
with open(labels_path, 'r') as file:
    data = json.load(file)

# Create Training and Validation Folders if they don't exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)

# List of image filenames
image_filenames = [img["file_name"] for img in data["images"]]

# Shuffle and split
random.shuffle(image_filenames)
split_idx = int(len(image_filenames) * 0.8)  # 80% train, 20% val

train_filenames = image_filenames[:split_idx]
val_filenames = image_filenames[split_idx:]

# Move Images
for filename in train_filenames:
    src = os.path.join(original_images_path, filename)
    dst = os.path.join(train_images_path, filename)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"File not found: {src}")

for filename in val_filenames:
    src = os.path.join(original_images_path, filename)
    dst = os.path.join(val_images_path, filename)
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        print(f"File not found: {src}")

# Update labels.json
train_data = {"images": [], "annotations": [], "categories": data["categories"]}
val_data = {"images": [], "annotations": [], "categories": data["categories"]}

train_image_ids = set()
val_image_ids = set()

for img in data["images"]:
    if img["file_name"] in train_filenames:
        train_data["images"].append(img)
        train_image_ids.add(img["id"])
    elif img["file_name"] in val_filenames:
        val_data["images"].append(img)
        val_image_ids.add(img["id"])

for ann in data["annotations"]:
    if ann["image_id"] in train_image_ids:
        train_data["annotations"].append(ann)
    elif ann["image_id"] in val_image_ids:
        val_data["annotations"].append(ann)

# Save updated JSON files
with open("android_figurine/Blue_ball/train_labels.json", 'w') as file:
    json.dump(train_data, file, indent=4)

with open("android_figurine/Blue_ball/val_labels.json", 'w') as file:
    json.dump(val_data, file, indent=4)



In [ ]:
train_data = object_detector.Dataset.from_coco_folder("android_figurine/Blue_ball/train/", cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(val_images_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)


In [26]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)


In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")


In [ ]:
model.export_model()

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")
# Quantization seems to significiantly reduce results! Skip quantization for the real model

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")


In [ ]:
model.export_model('model_int8_qat.tflite')

In [ ]:
from mediapipe_model_maker import quantization
quantization_config = quantization.QuantizationConfig.for_float16()
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)

In [ ]:
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
# The post-training quantization step seems to have no effect on the model evaluation, but it should make the model faster so ALWAYS include this step!